In [1]:
from pathlib import Path
path_spec_data=Path.cwd().parent.parent/"spec_data"
path_benchmark_data=Path.cwd().parent.parent/"benchmark_for_test"

path_spec_data.mkdir(parents=True, exist_ok=True)
path_benchmark_data.mkdir(parents=True, exist_ok=True)

In [ ]:
# Only record build and update in dynamic

library_size = 1_000_000

ion_mode = [-1, 
              1
            ] 

added_size=1_000_000
num_per_group=100_000_000
cache_list_threshold=1_000_000
mass_per_block=[0.05, 0.1, 0.5, 1]

steps=["build", 
       "update"
       ]
dynamic_script_path="16_dynamic_entropy_search_fast_update_options_mass_per_block.py"

In [ ]:
import subprocess
import shutil
import os
import numpy as np
from typing import Union
import time
def run_usrbintime_by_arguments(
          arguments:list[str], 
          if_output:bool=False, 
          output_memory_file:Union[str,Path]=None, 
          output_time_file:Union[str, Path]=None):
    
    # arguments: script_path, str(charge), step
    command=["/usr/bin/time","-v","python"] + arguments

    if if_output: # Output to files as record
        with open(output_memory_file, "w") as f1, open(output_time_file, "w") as f2:
            subprocess.run(command, stderr=f1, stdout=f2, cwd=Path.cwd(), env=os.environ.copy())

    else: # Output is not needed
         
        subprocess.run(command, stderr=subprocess.DEVNULL, stdout=subprocess.DEVNULL, cwd=Path.cwd(), env=os.environ.copy())
        
    return

for charge in ion_mode:
    for mass in mass_per_block:
        for step in steps:
            # Step 1: Read the query spectra, library spectra and added spectra 
            if step=="build":
                # For library spectra
                random_select_num=np.random.choice(100, 1, replace=False)
                spectra_bin=path_spec_data/f"35_random_export_ms2/charge_{charge}/batch_{random_select_num[0]}.bin"
                    
                path_comparison_dynamic_data=Path.cwd().parent.parent/f"comparison_data/dynamic/charge-{charge}"
                if path_comparison_dynamic_data.exists():
                    shutil.rmtree(path_comparison_dynamic_data)

                if_output=False
                output_memory_file=None
                output_time_file=None

            elif step=="update":
                # For insert spectra
                random_select_num=np.random.choice(100, 1, replace=False)
                spectra_bin=path_spec_data/f"35_random_export_ms2/charge_{charge}/batch_{random_select_num[0]}.bin"          
                
                if_output=True
                output_memory_file=path_benchmark_data/f"dynamic_fast_update_{charge}_{added_size}_added_to_{library_size}_memory_usage_{step}_step_options_{mass}_mass_per_block.txt"
                output_time_file=path_benchmark_data/f"dynamic_fast_update_{charge}_{added_size}_added_to_{library_size}_compare_time_{step}_step_options_{mass}_mass_per_block.txt"
            
            arguments=[dynamic_script_path, str(charge), str(num_per_group), str(cache_list_threshold), spectra_bin, str(mass), step]
            run_usrbintime_by_arguments(arguments=arguments,
                                        if_output=if_output,
                                        output_memory_file=output_memory_file,
                                        output_time_file=output_time_file)

                